#  Shadow Model

The adversary trains $k$ shadow models $f^i_{shadow}()$, each on a dataset (shadow data) that is similar
in format and distribution as the __target model's private training set__.

Generate shadow data drawing samples from the population where the target model's training data
are drawn from. The shadow model must be __trained in a similar fashion as the target model__.
The larger the number of shadow models, the better the accuracy of the attack model.

In [295]:
%run ../shadow_model.py

In [296]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GroupShuffleSplit, StratifiedShuffleSplit
from sklearn.datasets import load_wine

In [297]:
import numpy as np
import pandas as pd

In [298]:
data = pd.read_csv('synthetic_irir.csv')

In [299]:
data.head()

,0,1,2,3,label
0,0.495151,0.471551,0.113910,0.047197,0
1,0.083423,0.210501,0.006217,0.152407,0
2,0.694754,0.832320,0.222306,0.176190,0
3,0.620594,0.812440,0.244068,0.134671,0
4,0.625834,0.568340,0.177182,0.100158,0


In [300]:
rf =  RandomForestClassifier()

In [301]:
sh = ShadowModels(5, data, 3, rf)

In [304]:
sh.results[:3]

array([[0. , 0.1, 0.9, 2. , 1. ],
       [0. , 0. , 1. , 2. , 1. ],
       [0.8, 0.2, 0. , 0. , 1. ]])